# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_df = pd.read_csv("../weather.csv")
file_df.head()

,City,City_ID,Latitude,Longitutde,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Albany,5106841,42.6001,-73.9662,US,12.07,91,77,2.28,2022-03-20 02:53:07
1,Cherskiy,2126199,68.7500,161.3000,RU,-9.20,83,12,5.80,2022-03-20 02:43:47
2,Vardø,777019,70.3705,31.1107,NO,1.37,69,75,13.67,2022-03-20 02:49:12
3,Adrar,2381972,20.5022,-10.0711,MR,19.82,23,49,3.83,2022-03-20 02:43:49
4,Avarua,4035715,-21.2078,-159.7750,CK,28.03,69,51,2.57,2022-03-20 02:43:50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key = g_key)
# Store 'Lat' and 'Lng' into  locations 
locations_df = file_df[["Latitude", "Longitutde"]].astype(float)
locations_df.head()

humidity = file_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#My filters: >25C but less then <35, humidiity <70, wind speed <10, cloudiness < 30:
fab_holiday_df = file_df.loc[(file_df["Max Temp"]>25) & (file_df["Max Temp"]<35) \
                            & (file_df["Humidity"]<70) & (file_df["Wind Speed"] <10)\
                           & (file_df["Cloudiness"]<40)] 
new_df = fab_holiday_df.dropna()
new_df.shape

(20, 10)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = new_df[["City","Country", "Latitude", "Longitutde"]]
hotel_df["Hotel Name"] = ""
hotel_df

/var/folders/c1/_wjd1nqx03q7043fsjpcgjd80000gn/T/ipykernel_54795/728851477.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Latitude,Longitutde,Hotel Name
84,Gulariyā,NP,28.2333,81.3333,
96,Kwang Binh,VN,17.4833,106.6000,
98,Tura,IN,25.5198,90.2201,
158,Bilāri,IN,28.6333,78.8000,
199,Liancheng,CN,24.0519,105.0750,
217,Bafoulabé,ML,13.8065,-10.8321,
247,Goundi,TD,9.3627,17.3660,
295,Roebourne,AU,-20.7833,117.1333,
313,Port Macquarie,AU,-31.4333,152.9167,
317,Port Moresby,PG,-9.4431,147.1797,


In [16]:
target_type = "lodging"
radius = 10000

params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    params["location"]= f'{row["Latitude"]},{row["Longitutde"]}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    try:
        hotels = requests.get(base_url, params = params).json()
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]['name']
        
    except (KeyError, IndexError):
        print("Hotel not found")
hotel_df

Hotel not found


,City,Country,Latitude,Longitutde,Hotel Name
84,Gulariyā,NP,28.2333,81.3333,PUJA TRADING
96,Kwang Binh,VN,17.4833,106.6000,Nam Long Hotel
98,Tura,IN,25.5198,90.2201,Hotel RIKMAN Continental
158,Bilāri,IN,28.6333,78.8000,Klapsons Inn
199,Liancheng,CN,24.0519,105.0750,Kexiang Hotel
217,Bafoulabé,ML,13.8065,-10.8321,Logement du Chargé SIS Abdoul Karim Konaté
247,Goundi,TD,9.3627,17.3660,
295,Roebourne,AU,-20.7833,117.1333,Latitude20 Roebourne Village
313,Port Macquarie,AU,-31.4333,152.9167,Rydges Port Macquarie
317,Port Moresby,PG,-9.4431,147.1797,Lamana Hotel (Port Moresby)


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitutde"]]

In [14]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(hotel_marker)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))